In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
import keras.backend as K
from keras.layers import Dense, Input
from keras.models import Model
from scipy.ndimage import gaussian_filter
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches

Using TensorFlow backend.


In [2]:
ls /Users/dgagne/data/spatial_storm_results_20171220/

conv_net_best_params.csv
conv_net_param_combos.csv
conv_net_param_scores_sample_000.csv
conv_net_param_scores_sample_001.csv
conv_net_param_scores_sample_002.csv
conv_net_param_scores_sample_003.csv
conv_net_param_scores_sample_004.csv
conv_net_param_scores_sample_005.csv
conv_net_param_scores_sample_006.csv
conv_net_param_scores_sample_007.csv
conv_net_param_scores_sample_008.csv
conv_net_param_scores_sample_009.csv
conv_net_param_scores_sample_010.csv
conv_net_param_scores_sample_011.csv
conv_net_param_scores_sample_012.csv
conv_net_param_scores_sample_013.csv
conv_net_param_scores_sample_014.csv
conv_net_param_scores_sample_015.csv
conv_net_param_scores_sample_016.csv
conv_net_param_scores_sample_017.csv
conv_net_param_scores_sample_018.csv
conv_net_param_scores_sample_019.csv
conv_net_param_scores_sample_020.csv
conv_net_param_scores_sample_021.csv
conv_net_param_scores_sample_022.csv
conv_net_param_scores_sample_023.csv
conv_net_param_scores_sample_024.csv
conv_net_param_scores_sa

In [2]:
log_pca_files = sorted(glob("/glade/work/dgagne/spatial_storm_results_20171220/hail_logistic_pca_sample_*.pkl"))

In [3]:
label_spots_t = [(2, 2),
               (10, 10), (10, 26), 
               (20, 10), (20, 26)]
label_spots_d = [
               (5, 10), (5, 22), 
               (18, 12), (18, 28),
               (28, 28)]

In [8]:
samples = np.arange(0, 30)
for sample in samples:
    print(sample)
    with open(log_pca_files[sample], "rb") as pca_file:
        log_pca_obj = pickle.load(pca_file)
    k_log_input = Input((75,))
    k_log_out = Dense(1, activation="sigmoid")(k_log_input)
    k_mod = Model(k_log_input, k_log_out)
    k_mod.compile("sgd", loss="mse")
    k_mod.layers[1].set_weights([log_pca_obj.model.coef_.T, log_pca_obj.model.intercept_])
    loss = K.mean((k_mod.output - 1) ** 2)
    grad = K.gradients(loss, [k_mod.input])[0]
    grad /= K.maximum(K.std(grad), K.epsilon())
    grad_func = K.function([k_mod.input], [grad, loss])
    input_vals = np.zeros((1, 75))
    for i in range(10):
        g_val, l_val = grad_func([input_vals])
        print(l_val)
        input_vals -= 0.1 * g_val
    layer_vis = np.zeros((1, 32, 32, 15))
    for v in range(15):
        layer_vis[0, :, :, v] = log_pca_obj.pca[v].inverse_transform(input_vals[:, v * 5:(v+1)*5]).reshape(32, 32)
    fig = plt.figure(figsize=(12, 4))
    heights = [500, 700, 850]
    for a in np.arange(3):
        plt.subplot(1,3, a + 1)
        hght_g = gaussian_filter(layer_vis[0, :, :, a], 1)
        hght_g /= hght_g.std()
        hght = plt.contourf(np.arange(0, 32) + 0.5, np.arange(0, 32) + 0.5, hght_g, [-3, -2, -1, 1, 2, 3], extend="both", cmap="RdBu_r", zorder=0)
        tmpc_g = gaussian_filter(layer_vis[0, :, :, 3 + a], 1)
        tmpc_g /= tmpc_g.std()
        tmpc = plt.contour(np.arange(0, 32) + 0.5, np.arange(0, 32) + 0.5, tmpc_g, 
                           [-3, -2, -1, 1, 2, 3], linewidths=3, colors=["orangered"], zorder=0.1)
        if np.abs(tmpc_g).max() > 1: 
            plt.clabel(tmpc, fmt="%1.0f", fontsize=12, manual=label_spots_t)
        dewp_g = gaussian_filter(layer_vis[0, :, :, 6 + a], 1)
        dewp_g /= dewp_g.std()
        dewp = plt.contour(np.arange(0, 32) + 0.5, np.arange(0, 32) + 0.5,
                           dewp_g, [-3, -2, -1,1,2,3], linewidths=2, colors=["purple"], zorder=0.1)
        if np.abs(dewp_g).max() > 1:
            plt.clabel(dewp, fmt="%1.0f", fontsize=12, manual=label_spots_d)
        u_g = gaussian_filter(layer_vis[0, :, :, 9 + a], 1)
        u_g /= u_g.std()
        v_g = gaussian_filter(layer_vis[0, :, :, 12 + a], 1)
        v_g /= v_g.std()
        wind_mask = np.sqrt(u_g ** 2 + v_g ** 2) < 0.5
        u_g[wind_mask] = 0
        v_g[wind_mask] = 0
        qv = plt.quiver(np.arange(0, 32, 2) + 0.5, np.arange(0, 32, 2) + 0.5, u_g[::2, ::2],
                        v_g[::2, ::2], color="k", scale=64)
        plt.xticks(np.arange(0, 36, 8), np.arange(0, 36, 8) * 3)
        plt.yticks(np.arange(0, 36, 8), np.arange(0, 36, 8) * 3)
        plt.xlabel("West-East Distance (km)", fontsize=12)
        plt.ylabel("South-North Distance (km)", fontsize=12)
        plt.title("{0:d} hPa".format(heights[a]), fontsize=14)
        rect = patches.Rectangle((18, 0), 14, 6, facecolor='white', edgecolor='k', alpha=0.9, zorder=1)
        plt.gca().add_patch(rect)
        plt.text(19, 5, "Temperature", color='orangered', fontsize=10, fontweight="bold", ha="left", va="center", zorder=3)
        plt.text(19, 3, "Dewpoint", color="purple", fontsize=10, fontweight="bold", ha="left", va="center", zorder=3)
        plt.quiverkey(qv, 20, 1, 3, 'Wind (3 $\sigma$)', coordinates='data', labelpos='E')
        
    cax = fig.add_axes([0.91, 0.1, 0.02, 0.8])
    fig.colorbar(hght, cax=cax, label="Geopotential Height Anomaly")
    plt.suptitle("Logistic PCA Optimized Hailstorm Model {0:02d}".format(sample), fontsize=16, y=0.99)
    plt.savefig("./logistic_pca_best_hailstorm_{0:02d}.pdf".format(sample), bbox_inches="tight", dpi=300)
    plt.close()

0


/glade/u/apps/dav/opt/python/3.6.4/intel/17.0.1/pkg-library/20180801-DL/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator PCA from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/glade/u/apps/dav/opt/python/3.6.4/intel/17.0.1/pkg-library/20180801-DL/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


0.811726
0.60253894
0.32432315
0.112430505
0.026028171
0.004670861
0.0007405331
0.00011140738
1.6414706e-05
2.3990485e-06
1
0.7956542
0.5912344
0.32806012
0.12299678
0.031943563
0.006498314
0.0011642411
0.00019693233
3.2509623e-05
5.3134413e-06
2
0.82901067
0.61857045
0.3266377
0.10606852
0.022157073
0.0035466412
0.0005024635
6.780406e-05
8.98404e-06
1.1824978e-06
3
0.8155643
0.58706695
0.28669482
0.08319363
0.015567772
0.0022793734
0.00030022734
3.7983315e-05
4.7356966e-06
5.873636e-07
4
0.8090329
0.6121765
0.34935158
0.13502984
0.035834126
0.007361639
0.001321436
0.00022307785
3.669115e-05
5.970946e-06
5
0.820409
0.62160677
0.34921822
0.12888743
0.03182698
0.0060289223
0.0009988914
0.00015612009
2.3829052e-05
3.6028214e-06
6
0.8196742
0.60181904
0.3089128
0.09705811
0.019779917
0.003122682
0.00043921304
5.901626e-05
7.795601e-06
1.0232342e-06
7
0.7977882
0.5701551
0.2830053
0.0872099
0.017909626
0.002904439
0.00042308975
5.9032744e-05
8.102445e-06
1.1053711e-06
8
0.7877181
0.5714335
